### *IT3052E - Fundamentals of Optimization*
# **Mini Project 18 - Nurse Scheduling Problem**
#### **Techniques used**:
* Backtracking,
* Constraint Programming,
* Linear Programming,
* Local Search, and
* Meta-heuristics (Genetic Algorithm).

#### * ***Import pandas for printing solution***

In [2]:
import pandas as pd

## 3. Hill-climbing search

### 3.1. Import modules and libraries

In [1]:
from PyCBLS.VarIntLS import VarIntLS
from PyCBLS.LocalSearchManager import LocalSearchManager
from PyCBLS.NotEqual import NotEqual
from PyCBLS.ConstraintSystem import ConstraintSystem
from PyCBLS.HillClimbingSearch import HillClimbingSearch
from PyCBLS.ConditionalSumWithBound2 import ConditionalSumWithBound
import random as rd

### 3.2. Read data from files

In [3]:
with open('/Users/ngocminhta/Optimization-NSP/SampleData/testCase1/0.txt') as file:
  N, D, a, b = [int(q) for q in file.readline().split()]
  dayoff = [[0 for d in range(D)] for n in range(N)]
  for n in range(N):
    for d in [int(h) for h in file.readline().split()]:
      if d != -1:
            dayoff[n][d-1] = 1

### 3.3. Generate local search and define decision variable

In [4]:
mgr = LocalSearchManager()

LocalSearchManagerconstructor


In [5]:
assign = [[VarIntLS(mgr,0,4,rd.randint(1, 4),f'assign_{n}_{d}') for n in range(N)] for d in range(D)]

mor = VarIntLS(mgr,1,1,1,'one')
aft = VarIntLS(mgr,2,2,2,'one')
eve = VarIntLS(mgr,3,3,3,'one')
nig = VarIntLS(mgr,4,4,4,'one')
zero = VarIntLS(mgr,0,0,0,'zero')

### 3.4. Generate constraints

In [6]:
constraints = []

#### 3.4.1. A nurse can be assigned to only one shift per day.

This is constrained by creating decision variable

#### 3.4.2. Each shift has min $a$ nurses and max $b$ nurses.

In [7]:
count = [1 for i in range(N)]
for d in range(D):
    c = ConditionalSumWithBound(assign[d],count,1,a,b,'CountMor')
    constraints.append(c)
    c = ConditionalSumWithBound(assign[d],count,2,a,b,'CountAft')
    constraints.append(c)
    c = ConditionalSumWithBound(assign[d],count,3,a,b,'CountEve')
    constraints.append(c)
    c = ConditionalSumWithBound(assign[d],count,4,a,b,'CountNig')
    constraints.append(c)

#### 3.4.3. Set the given dayoff and dayoff after a night shift

In [8]:
for n in range(N):
    for d in range(D-1):
        if dayoff[n][d] == 1 or assign[d-1][n] == nig:
            c = NotEqual(assign[d][n],mor,'NotEqual')
            constraints.append(c)
            c = NotEqual(assign[d][n],aft,'NotEqual')
            constraints.append(c)
            c = NotEqual(assign[d][n],eve,'NotEqual')
            constraints.append(c)
            c = NotEqual(assign[d][n],nig,'NotEqual')
            constraints.append(c)
        else:
            c = NotEqual(assign[d][n],zero,'NotEqual')
            constraints.append(c)
            
    # handle the last day
    if dayoff[n][D-1] == 1:
        c = NotEqual(assign[D-1][n],mor,'NotEqual')
        constraints.append(c)
        c = NotEqual(assign[D-1][n],aft,'NotEqual')
        constraints.append(c)
        c = NotEqual(assign[D-1][n],eve,'NotEqual')
        constraints.append(c)
        c = NotEqual(assign[D-1][n],nig,'NotEqual')
        constraints.append(c)
    else:
        c = NotEqual(assign[D-1][n],zero,'NotEqual')
        constraints.append(c)

### 3.5. Start local search

In [16]:
C = ConstraintSystem(constraints)
mgr.close()

print('Init, C = ',C.violations())
searcher = HillClimbingSearch(C)
searcher.search(1000)

ConstraintSystem::constructor
LocalSearchManager::close
topoSortInvariants finished
Init, C =  1
0 : assign x[ 24 ] =  1  violations =  1
1 : assign x[ 41 ] =  2  violations =  1
2 : assign x[ 55 ] =  0  violations =  1
3 : assign x[ 59 ] =  1  violations =  1
4 : assign x[ 52 ] =  0  violations =  1
5 : assign x[ 25 ] =  4  violations =  1
6 : assign x[ 48 ] =  3  violations =  1
7 : assign x[ 28 ] =  0  violations =  1
8 : assign x[ 24 ] =  3  violations =  1
9 : assign x[ 57 ] =  4  violations =  1
10 : assign x[ 2 ] =  1  violations =  1
11 : assign x[ 36 ] =  2  violations =  1
12 : assign x[ 15 ] =  4  violations =  1
13 : assign x[ 54 ] =  2  violations =  1
14 : assign x[ 16 ] =  2  violations =  1
15 : assign x[ 63 ] =  3  violations =  1
16 : assign x[ 49 ] =  1  violations =  1
17 : assign x[ 17 ] =  3  violations =  1
18 : assign x[ 53 ] =  2  violations =  1
19 : assign x[ 2 ] =  4  violations =  1
20 : assign x[ 35 ] =  1  violations =  1
21 : assign x[ 59 ] =  2  violati

### 3.6. Print the schedule

In [17]:
res = [[0 for n in range(N)] for d in range(D)]
for d in range(D):
    for n in range(N):
        res[d][n] = assign[d][n].getValue()
df = pd.DataFrame(res, index = [d+1 for d in range(D)], columns = [n+1 for n in range(N)])
df.index.name = 'Day'
df.columns.name = 'Nurse'
display(df)

Nurse,1,2,3,4,5,6,7,8,9,10
Day,,,,,,,,,,
1,1,1,2,0,4,1,0,1,1,1
2,3,1,0,3,0,0,1,1,0,2
3,4,3,2,0,4,0,3,3,3,4
4,2,0,3,2,0,3,0,4,4,4
5,0,1,3,2,3,0,2,2,0,2
6,2,0,2,2,0,2,3,1,2,3


### 3.7. Optimal solution

In [14]:
maxNightShift = -1
for n in range(N):
    tmp = 0
    for d in range(D):
        if res[d][n] == 4:
            tmp += 1
    if tmp > maxNightShift:
        maxNightShift = tmp
        
print('Optimal solution - Max night shift assigned to a nurse:', maxNightShift)

Optimal solution - Max night shift assigned to a nurse: 2
